# 合并ksrpc+frpc两阻塞代码于单Notebook

In [ ]:
def run_clear():
    print("启动定时清屏")
    
    import time
    from IPython.display import clear_output
    
    while True:
        time.sleep(30)
        clear_output(wait=True)

In [ ]:
def run_frpc():
    print("启动 frpc 服务...")
    
    import subprocess
    try:
        # 使用 subprocess 运行外部命令
        process = subprocess.Popen(
            ["./frpc", "-c", "./frpc.toml"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
        )
        
        # 实时输出日志
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if output:
                print(output.strip())
                
        # 等待进程结束
        process.wait()
        
    except Exception as e:
        print(f"frpc 服务启动失败: {e}")
    finally:
        print("frpc 服务已终止")

In [ ]:
def run_ksrpc():
    print("启动 ksrpc 服务")
    
    from ksrpc import config
    config.PORT = 8000
    
    # 必须在主线程中使用，否则调用的部分API可能失败。所以要线程中使用
    from ksrpc.app import simple_
    simple_.main()

In [ ]:
import multiprocessing
from contextlib import contextmanager

@contextmanager
def process_manager(*processes):
    try:
        for p in processes:
            p.start()
        yield
    finally:
        for p in processes:
            if p.is_alive():
                p.terminate()
                p.join()

if __name__ == '__main__':
    p1 = multiprocessing.Process(target=run_ksrpc)
    p2 = multiprocessing.Process(target=run_frpc)
    p3 = multiprocessing.Process(target=run_clear)
    
    with process_manager(p1, p2, p3):
        p1.join()
        p2.join()
        p3.join()